I chose to make a model that takes in 5 years of stock data and will decide whether to buy, sell, or hold that stock position.

I am comparing a baseline Linear NN with two more advanced models to see whether predictions and accuracy improve with more complexity.

In [34]:
%pip install pandas numpy yfinance scikit-learn torch matplotlib seaborn

Note: you may need to restart the kernel to use updated packages.


In [35]:
import pandas as pd
import numpy as np
import yfinance as yf
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import os

# For prettier plots
plt.style.use('fivethirtyeight')
sns.set_theme(style="darkgrid")

from enhanced_stock_model import (AttentionLSTM, prepare_data, train_model,
                                 evaluate_model, save_model, load_model,
                                 predict_next_day)

# Choose Stock and Download Data

In [36]:
config = {
    'ticker': 'TSLA',           # Stock to predict
    'benchmark': '^GSPC',       # Market benchmark (S&P 500)
    'start_date': '2015-01-01', # Training data start date (5+ years for best results)
    'seq_length': 30,           # How many trading days to use for each prediction
    'hidden_size': 128,         # LSTM hidden layer size
    'num_layers': 3,            # Number of stacked LSTM layers
    'dropout': 0.4,             # Dropout rate for regularization
    'learning_rate': 0.0005,    # Learning rate for optimizer
    'batch_size': 64,           # Batch size for training
    'num_epochs': 200,          # Maximum number of epochs
    'patience': 20              # Early stopping patience
}

In [40]:
X_train, y_train, X_test, y_test, scaler, feature_names, close_prices, dates = prepare_data(
    ticker=config['ticker'],
    benchmark=config['benchmark'],
    start_date=config['start_date'],
    seq_length=config['seq_length']
)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [42]:
# Check the shape of our data
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")
print(f"Number of features: {X_train.shape[2]}")

X_train shape: (1948, 30, 42)
y_train shape: (1948,)
X_test shape: (487, 30, 42)
y_test shape: (487,)
Number of features: 42


# Setup Dataloaders

In [ ]:
features = ['Open', 'High', 'Low', 'Close', 'Volume', 'SP500_Close']
scaler = StandardScaler()
X = scaler.fit_transform(data[features])
y = data['Label'].values

X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)


batch_size = 1024
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=batch_size, shuffle=False)


NameError: name 'StandardScaler' is not defined

# Baseline Linear Neural Network Training

In [ ]:
class BaseLineNN(nn.Module):
    def __init__(self,  input_dim, output_dim):
        super(BaseLineNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 16)
        self.fc2 = nn.Linear(16, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

input_dim = len(features)
output_dim = 3
baseline_model = BaseLineNN(input_dim, output_dim)

baseline_criterion = nn.CrossEntropyLoss()
baseline_optimizer = optim.SGD(baseline_model.parameters(), lr=0.01)

In [ ]:
num_epochs = 1000
best_baseline_loss = float("inf")
baseline_model_path = "best_baseline_model.pth"

for epoch in range(num_epochs):
    baseline_model.train()
    total_loss = 0
    correct_train = 0
    total_train = 0

    for inputs, labels in train_loader:
        baseline_optimizer.zero_grad()
        outputs = baseline_model(inputs)
        loss = baseline_criterion(outputs, labels + 1)
        loss.backward()
        baseline_optimizer.step()
        total_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        correct_train += (predicted == (labels + 1)).sum().item()
        total_train += labels.size(0)

    train_accuracy = correct_train / total_train

    baseline_model.eval()
    total_val_loss = 0
    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = baseline_model(inputs)
            val_loss = baseline_criterion(outputs, labels + 1)
            total_val_loss += val_loss.item()

            _, predicted = torch.max(outputs, 1)
            correct_test += (predicted == (labels + 1)).sum().item()
            total_test += labels.size(0)

    val_accuracy = correct_test / total_test
    avg_train_loss = total_loss / len(train_loader)
    avg_val_loss = total_val_loss / len(test_loader)

    if avg_val_loss < best_baseline_loss:
        best_baseline_loss = avg_val_loss
        torch.save(baseline_model.state_dict(), baseline_model_path)

    if epoch % 10 == 0:
        print(f"Baseline Epoch [{epoch+1}/{num_epochs}] | "
              f"Train Loss: {avg_train_loss:.4f} | Train Acc: {train_accuracy:.4%} | "
              f"Test Loss: {avg_val_loss:.4f} | Test Acc: {val_accuracy:.4%}")

print(f"\nBest baseline model saved to: {baseline_model_path} with Test Loss: {best_baseline_loss:.4f}")

# Compare and Evalutate both models

In [ ]:
def evaluate_model(model, test_loader, name="Model"):
    model.eval()
    predictions = []
    actuals = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.numpy())
            actuals.extend((labels + 1).numpy())  # Shift back from (-1,0,1) to (0,1,2)

    print(f"{name} Classification Report:")
    print(classification_report(actuals, predictions, target_names=['Sell', 'Hold', 'Buy']))
    return predictions, actuals

# Evaluate both models
predictions_baseline, actuals_baseline = evaluate_model(baseline_model, test_loader, name="Baseline NN")
predictions_baseline, actuals_baseline = evaluate_model(mediumClassifier, test_loader, name="Baseline NN")
predictions_advanced, actuals_advanced = evaluate_model(improved_model, test_loader, name="Advanced NN")

In [ ]:
latest_data = stock_data.iloc[-1][['Open', 'High', 'Low', 'Close', 'Volume']]
latest_sp500 = sp500_data.iloc[-1]['Close']

latest_features = np.array([latest_data['Open'], latest_data['High'], latest_data['Low'], latest_data['Close'], latest_data['Volume'], latest_sp500])
latest_features = latest_features.reshape(1, -1)
latest_features_scaled = scaler.transform(latest_features)
latest_tensor = torch.tensor(latest_features_scaled, dtype=torch.float32)

baseline_model.eval()
with torch.no_grad():
    output = baseline_model(latest_tensor)
    _, predicted_class = torch.max(output, 1)

label_map = {0: "Sell", 1: "Hold", 2: "Buy"}
predicted_label = label_map[predicted_class.item()]

print(f"Baseline model: Today's recommended action for {stock_ticker}: {predicted_label}")